In [1]:
import numpy as np
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (
     FigureCanvasTkAgg)
import tkinter as tk
from tkinter import messagebox

# File paths
files = ['/Users/divyanshukumar/Downloads/m&m.csv', '/Users/divyanshukumar/Downloads/itc.csv', '/Users/divyanshukumar/Downloads/l&t.csv', '/Users/divyanshukumar/Downloads/hdfc.csv', '/Users/divyanshukumar/Downloads/sunpha.csv', '/Users/divyanshukumar/Downloads/tcs.csv']
dfs = []

for file in files:
    temp = pd.read_csv(file)
    temp.columns = ['Date', file.replace('.csv', '')]
    dfs.append(temp)

stocks = reduce(lambda left, right: pd.merge(left, right, on='Date'), dfs)

# Calculate returns for the last 12 months
returns_12_months = (stocks.iloc[0, 1:] - stocks.iloc[12, 1:]) / stocks.iloc[12, 1:]

# Ant Colony Optimization for Portfolio Optimization
np.random.seed(0)

# Define parameters
num_assets = returns_12_months.shape[0]
num_ants = 20
num_iterations = 100
rho = 0.1  # Pheromone evaporation rate
alpha = 1.0  # Pheromone attractiveness parameter
beta = 2.0  # Heuristic information attractiveness parameter
evaporation_iterations = 10  # Evaporation after every n iterations
evaporation_rate = 0.5  # Pheromone evaporation factor

# Initialize pheromones
pheromones = np.ones((num_assets, num_assets)) / num_assets

# Define the fitness function
def fitness(weights, returns):
    portfolio_return = np.sum(returns * weights)
    return portfolio_return

# Main Ant Colony Optimization loop
for iteration in range(num_iterations):
    ant_weights = np.zeros((num_ants, num_assets))
    ant_returns = np.zeros(num_ants)

    # Generate solutions for each ant
    for ant in range(num_ants):
        current_asset = np.random.randint(num_assets)
        selected_assets = [current_asset]

        for _ in range(num_assets - 1):
            # Calculate transition probabilities
            probabilities = pheromones[current_asset] ** alpha

            # Set already selected assets' probabilities to 0
            probabilities[selected_assets] = 0

            probabilities /= np.sum(probabilities)

            # Select the next asset based on the probabilities
            next_asset = np.random.choice(np.arange(num_assets), p=probabilities)
            selected_assets.append(next_asset)
            current_asset = next_asset

        # Normalize weights based on the selected assets
        ant_weights[ant, selected_assets] = np.random.rand(len(selected_assets))
        ant_weights[ant] /= np.sum(ant_weights[ant])  # Normalize the weights

        # Evaluate the ant's portfolio
        ant_returns[ant] = fitness(ant_weights[ant], returns_12_months)

    # Update pheromones based on ant solutions
    delta_pheromones = np.zeros((num_assets, num_assets))
    for ant in range(num_ants):
        for i in range(num_assets - 1):
            delta_pheromones[selected_assets[i], selected_assets[i + 1]] += ant_returns[ant]

    pheromones = (1 - rho) * pheromones + delta_pheromones / np.sum(ant_returns)

    # Evaporate pheromones after every n iterations
    if iteration % evaporation_iterations == 0:
        pheromones *= (1 - evaporation_rate)

# Find the best portfolio
best_portfolio = ant_weights[np.argmax(ant_returns)]

root = tk.Tk()
root.title("Portfolio Optimization Results")

# Create a frame to hold the graph and printed output
frame = tk.Frame(root)
frame.pack(padx=20, pady=20)

# Plotting the graph of portfolio weights and returns
fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot portfolio weights as a bar chart
ax1.bar(range(num_assets), best_portfolio, align='center', alpha=0.7)
ax1.set_xlabel('Asset')
ax1.set_ylabel('Weight', color='tab:blue')
ax1.tick_params('y', colors='tab:blue')

# Create a second y-axis for returns
ax2 = ax1.twinx()
ax2.plot(range(num_assets), returns_12_months, color='tab:red', marker='o')
ax2.set_ylabel('Returns', color='tab:red')
ax2.tick_params('y', colors='tab:red')

plt.title('Best Portfolio Weights and Returns (12 months)')

# Display the printed output in a label
result_text = f"Best Portfolio Weights:\n{best_portfolio}\nExpected Return (12 months): {np.sum(returns_12_months * best_portfolio)}"
output_label = tk.Label(frame, text=result_text, justify="left", padx=10, pady=10)
output_label.pack()

# Embed the Matplotlib plot in the Tkinter window
canvas = FigureCanvasTkAgg(fig, master=frame)
canvas.draw()
canvas.get_tk_widget().pack()

root.mainloop()

# Call the function to display portfolio results

2023-11-18 18:38:40.852 Python[12127:567218] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 